In [1]:
#import necessary packages
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
from sklearn import metrics
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import keras
from tensorflow.keras import layers

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#read data and split labels from values
initial = (pd.read_csv('/content/drive/My Drive/GSE99095_normalizedExpression.csv',header=None)).T
#create labels
initial[0]=np.zeros(980)
for i in range(1,392):
    (initial[0].values)[i]=1

initial=initial.iloc[1:]
labels= initial[0].values
values = (initial.drop((initial[0]),axis=1)).values

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

In [3]:
#randomly shuffle data and do a train-test split on values and labels
values, labels = shuffle(values, labels,random_state=1) 
values_train = values[:700, :]
values_test = values[700:, :]
y_train = labels[:700]
y_test = labels[700:]

In [4]:
#build the autoencoder
inputs = keras.Input(values.shape[1],)
b1 = layers.BatchNormalization(name="batch1")(inputs)
lr1 = layers.Dense(8192, activation="softplus", name="lr1")(b1)
b2 = layers.BatchNormalization(name="batch2")(lr1)
lr2 = layers.Dense(1024, activation="softplus", name="lr2")(b2)
b3 = layers.BatchNormalization(name="batch3")(lr2)
lr3 = layers.Dense(128, activation="softplus", name="lr3")(b3)
b4 = layers.BatchNormalization(name="batch4")(lr3)
lr4 = layers.Dense(2048, activation="softplus", name="lr4")(b4)
b5 = layers.BatchNormalization(name="batch5")(lr4)
outputs = layers.Dense(values.shape[1], activation="softplus", name="outputs")(b5)

#build a seperate encoder for feature extraction
encoder = keras.models.Model(inputs, lr3)

#connect the autoencoder
stacked_ae = keras.models.Model(inputs,outputs)

#compile the model
stacked_ae.compile(loss="mean_squared_error",optimizer=keras.optimizers.Nadam(learning_rate=0.0001))

#train the model
h_stack = stacked_ae.fit(values_train, values_train, epochs=200,batch_size=8,validation_data=[values_test, values_test])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 700 samples, validate on 279 samples
Epoch 1/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.5821 - val_loss: 2.5691
Epoch 2/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.5328 - val_loss: 2.5206
Epoch 3/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.4983 - val_loss: 2.4748
Epoch 4/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.4530 - val_loss: 2.4188
Epoch 5/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.3884 - val_loss: 2.3608
Epoch 6/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.3016 - val_loss: 2.2910
Epoch 7/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.2208 - val_loss: 2.2350
Epoch 8/200
700/700 [==============================] - 23s 33ms/sample - loss: 2.1532 - val_loss: 2.1770
Epoch 9/200
700/700 [============

In [5]:
#extract the features for whole data using trained model
features= encoder.predict(values)

In [7]:
#randomly shuffle data and do a train-test split on values and labels using the new features
features, labels = shuffle(features, labels,random_state=1) 
X_train = features[:700, :]
X_test = features[700:, :]
y_train = labels[:700]
y_test = labels[700:]

#build the fully-connected model
fullyConnected = keras.models.Sequential([
  layers.Dense(256, activation='relu', input_shape=(128,)),
  layers.Dense(64, activation='relu'),
  layers.Dense(16, activation='relu'),
  layers.Dense(2, activation='softmax'),
])

#compile the model
fullyConnected.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.0001),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

#train the model
fullyConnected.fit(X_train, keras.utils.to_categorical(y_train), epochs=200,batch_size=8,)

#test the model and make final predictions
fullyConnected.evaluate(
  X_test,
  keras.utils.to_categorical(y_test)
)

Train on 700 samples
Epoch 1/200
700/700 [==============================] - 0s 360us/sample - loss: 0.7418 - acc: 0.6029
Epoch 2/200
700/700 [==============================] - 0s 415us/sample - loss: 0.5270 - acc: 0.7357
Epoch 3/200
700/700 [==============================] - 0s 378us/sample - loss: 0.3631 - acc: 0.8471
Epoch 4/200
700/700 [==============================] - 0s 385us/sample - loss: 0.2696 - acc: 0.9057
Epoch 5/200
700/700 [==============================] - 0s 381us/sample - loss: 0.2007 - acc: 0.9486
Epoch 6/200
700/700 [==============================] - 0s 387us/sample - loss: 0.1558 - acc: 0.9671
Epoch 7/200
700/700 [==============================] - 0s 358us/sample - loss: 0.1264 - acc: 0.9814
Epoch 8/200
700/700 [==============================] - 0s 376us/sample - loss: 0.1020 - acc: 0.9871
Epoch 9/200
700/700 [==============================] - 0s 379us/sample - loss: 0.0862 - acc: 0.9871
Epoch 10/200
700/700 [==============================] - 0s 405us/sample - loss:

[0.5273665877052426, 0.94265234]